In [1]:
import os
import glob

import concurrent.futures
import multiprocessing

import numpy as np
from astropy.io import fits
from skimage import io

from tqdm.notebook import tqdm


from sunscc.dataset.utils import create_circular_mask, get_sun_mask

In [2]:
img_dir = '../../datasets/clasiffication/2002-2019_2/image'
out_dir = '../../datasets/clasiffication/2002-2019_2/sun_mask'
# img_dir = '../../datasets/clasiffication/2002-2019/image'
# out_dir = '../../datasets/clasiffication/2002-2019/sun_mask'

In [10]:

num_cpu = multiprocessing.cpu_count()
# num_cpu = 1

def process_image(img_path, dest_dir):
    
    hdulst:fits.HDUList = fits.open(img_path)
    image = hdulst[0]
    header = image.header
    center = np.array(image.shape)//2
#     print(f'shape: {image.data.shape} , center: {center} {type(image.data)}')
#     print()
    
    radius = header['SOLAR_R']
    # sample['solar_disk'] = create_circular_mask( image.shape[0], image.shape[1] ,center,radius)
    basic_mask = create_circular_mask( image.shape[0], image.shape[1] ,center,radius=radius*1.03)
    solar_disk = get_sun_mask( image.data, basic_mask, radius)

    # save the mask in the destination directory
    filename = os.path.basename(img_path).replace('.FTS', '.png')
    dest_path = os.path.join(dest_dir, filename)

#     print(solar_disk.shape, np.unique(solar_disk))
    io.imsave(dest_path, solar_disk.astype(np.uint8), check_contrast=False)
    
# images = sorted(glob.glob(os.path.join(img_dir, '*.FTS')))[0:1000]
# images = sorted(glob.glob(os.path.join(img_dir, '*.FTS')))[1000:2000]
# images = sorted(glob.glob(os.path.join(img_dir, '*.FTS')))[2000:]
images = sorted(glob.glob(os.path.join(img_dir, '*.FTS')))[:]

with concurrent.futures.ProcessPoolExecutor(max_workers=num_cpu) as executor:
    for result in tqdm(executor.map(process_image, images, 
                                             [out_dir]*len(images))):
        pass


0it [00:00, ?it/s]